In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk.classify.scikitlearn import SklearnClassifier
import nltk
import random
import json
import pickle
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/NLTK_classification_Resume/Resume_Dataset.csv")

In [4]:
df.count()

Unnamed: 0    10172
text          10056
classe        10172
dtype: int64

In [5]:
df["classe"].unique()

array(['Education', 'Skills', 'Experience', 'Title', 'Certifications',
       'Location', 'Other'], dtype=object)

In [6]:
filter_ed = df["classe"]== "Education"
filter_sk = df["classe"]== "Skills"
filter_ex = df["classe"]== "Experience"

df = df.where(filter_ed | filter_sk | filter_ex).dropna()

In [7]:
df["classe"].unique()

array(['Education', 'Skills', 'Experience'], dtype=object)

In [8]:
df.count()

Unnamed: 0    5923
text          5923
classe        5923
dtype: int64

In [9]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
df["classe"] = df["classe"].str.lower()
df["text"] = df["text"].str.lower()

In [11]:
df

,Unnamed: 0,text,classe
0,0.0,in criminal justicetexas state university jan...,education
1,1.0,post graduate certificate in computer scienced...,education
2,10.0,associate,education
3,100.0,high school or equivalent in quality assurance...,education
4,1000.0,"in comptia a+ coursework, comptia network+ co...",education
...,...,...,...
9832,995.0,"skillsr, python, sas, c#, java, c++, c, adobe ...",skills
9833,996.0,"technical skillsprogramming languages: sql, vb...",skills
9834,997.0,• proven time management and organizational sk...,skills
9835,998.0,"technical skillsmachine learning: regression, ...",skills


In [12]:
all_words = []
stop_words = stopwords.words('english')
for text in df['text']:
    for word in word_tokenize(str(text)):
        if word.isalpha():
            if word not in stop_words:
                all_words.append(word)

In [13]:
records = df.to_records(index=False)
dataset = list(records)
dataset[0]

(0., ' in criminal justicetexas state university january 1995 to january 1999', 'education')

In [14]:
del(df)

In [15]:
len(all_words)

581664

In [16]:
len(dataset)

5923

In [17]:
all_words= FreqDist(all_words)

In [18]:
word_features = list(all_words.keys())

In [19]:
len(word_features)

42803

In [20]:
def get_features(document,word_features):
    words = set(word_tokenize(str(document)))
    words = [word_lower.lower() for word_lower in words]
    features = {}
    for word in word_features:
        features[word] = word in words
    return features

In [21]:
random.shuffle(dataset)

In [22]:
with open('/content/drive/MyDrive/Colab Notebooks/NLTK_classification_Resume/word_features.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(word_features, f)

In [23]:
features_set_training = [(get_features(text,word_features), classe) for (_,text,classe) in dataset[:4000]]

In [24]:
features_set_test = [(get_features(text,word_features), classe) for (_,text,classe) in dataset[4000:]]

In [25]:
with open('/content/drive/MyDrive/Colab Notebooks/NLTK_classification_Resume/features_set_training.pickle', 'wb') as f:
    pickle.dump(features_set_training, f)
with open('/content/drive/MyDrive/Colab Notebooks/NLTK_classification_Resume/features_set_test.pickle', 'wb') as f:
    pickle.dump(features_set_test, f)

In [27]:
del(dataset)

In [28]:
features_set_test[0][0]

{'criminal': False,
 'justicetexas': False,
 'state': False,
 'university': False,
 'january': False,
 'post': False,
 'graduate': False,
 'certificate': False,
 'computer': False,
 'sciencedepaul': False,
 'chicago': False,
 'ilbs': False,
 'marketingsouthern': False,
 'illinois': False,
 'college': False,
 'business': False,
 'carbondale': False,
 'il': False,
 'associate': False,
 'high': False,
 'school': False,
 'equivalent': False,
 'quality': False,
 'assurance': False,
 'engineeryear': False,
 'seattle': False,
 'wamarch': False,
 'october': False,
 'comptia': False,
 'coursework': False,
 'courseworknew': False,
 'horizons': False,
 'white': False,
 'plains': False,
 'nyjanuary': False,
 'science': False,
 'magnetgorton': False,
 'yonkers': False,
 'mor': False,
 'june': False,
 'diplomadelaware': False,
 'valley': False,
 'regional': False,
 'september': False,
 'scienceraritan': False,
 'community': False,
 'bachelor': False,
 'sciencecolorado': False,
 'mesa': False,
 'gran

# ***Load features set***

In [4]:
# load the model from disk
features_set_training = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/NLTK_classification_Resume/features_set_training.pickle', 'rb'))
# load the model from disk
features_set_test = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/NLTK_classification_Resume/features_set_test.pickle', 'rb'))


In [5]:
features_set_test[0][0]

{'criminal': False,
 'justicetexas': False,
 'state': False,
 'university': True,
 'january': True,
 'post': True,
 'graduate': True,
 'certificate': False,
 'computer': False,
 'sciencedepaul': False,
 'chicago': False,
 'ilbs': False,
 'marketingsouthern': False,
 'illinois': False,
 'college': True,
 'business': False,
 'carbondale': False,
 'il': False,
 'associate': False,
 'high': False,
 'school': False,
 'equivalent': False,
 'quality': False,
 'assurance': False,
 'engineeryear': False,
 'seattle': False,
 'wamarch': False,
 'october': False,
 'comptia': False,
 'coursework': False,
 'courseworknew': False,
 'horizons': False,
 'white': False,
 'plains': False,
 'nyjanuary': False,
 'science': False,
 'magnetgorton': False,
 'yonkers': False,
 'mor': False,
 'june': False,
 'diplomadelaware': False,
 'valley': False,
 'regional': False,
 'september': False,
 'scienceraritan': False,
 'community': False,
 'bachelor': False,
 'sciencecolorado': False,
 'mesa': False,
 'grand': F

# **Original Naive Bayes Classifier**

In [33]:
classifier = nltk.NaiveBayesClassifier.train(features_set_training)
print("Naive Bayes Classifier accuracy : " , (nltk.classify.accuracy(classifier,features_set_test) * 100),"%")

Naive Bayes Classifier accuracy :  91.99167966718669 %


In [35]:
classifier.show_most_informative_features(15)

Most Informative Features
                 january = True           experi : skills =    433.4 : 1.0
                    year = True           skills : educat =    382.5 : 1.0
                bachelor = True           educat : skills =    287.5 : 1.0
                  server = True           experi : educat =    248.0 : 1.0
                    work = True           experi : educat =    243.5 : 1.0
                hardware = True           experi : educat =    215.4 : 1.0
                 october = True           experi : skills =    207.3 : 1.0
                     sql = True           skills : educat =    207.2 : 1.0
                    java = True           skills : educat =    179.2 : 1.0
                services = True           experi : educat =    173.8 : 1.0
             environment = True           experi : educat =    169.9 : 1.0
                 working = True           experi : educat =    167.4 : 1.0
                       c = True           skills : educat =    166.6 : 1.0

In [40]:
classifier.classify(features_set_test[-1][0]), features_set_test[-1][1]

('experience', 'experience')

In [36]:
classifier.classify(features_set_test[2][0]), features_set_test[2][1]

('skills', 'skills')

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/NLTK_classification_Resume/NBclassifier.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(classifier, f, pickle.HIGHEST_PROTOCOL)

# **Multinomial, Gaussian and Bernoulli Naive Bayes Classifier**

In [6]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB

In [ ]:
MNBClassifier = SklearnClassifier(MultinomialNB())
MNB_classifier = MNBClassifier.train(features_set_training)
print("Multinomial Naive Bayes Classifier accuracy : " , (nltk.classify.accuracy(MNB_classifier,features_set_test) * 100),"%")

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/NLTK_classification_Resume/MNB_classifier.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(MNB_classifier, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
GNBClassifier = SklearnClassifier(GaussianNB())
GNB_classifier = GNBClassifier.train(features_set_training)
print("Gaussian Naive Bayes Classifier accuracy : " , (nltk.classify.accuracy(GNB_classifier,features_set_test) * 100),"%")

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/NLTK_classification_Resume/GNB_classifier.pickle', 'wb') as f:
  # Pickle the 'data' dictionary using the highest protocol available.
  pickle.dump(GNB_classifier, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
BNBClassifier = SklearnClassifier(BernoulliNB())
BNB_classifier = BNBClassifier.train(features_set_training)
print("Multinomial Naive Bayes Classifier accuracy : " , (nltk.classify.accuracy(BNB_classifier,features_set_test) * 100),"%")

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/NLTK_classification_Resume/BNB_classifier.pickle', 'wb') as f:
  # Pickle the 'data' dictionary using the highest protocol available.
  pickle.dump(BNB_classifier, f, pickle.HIGHEST_PROTOCOL)

# **Linear Model Classification : Logistic Regression, Stochastic Descent Gradient**

In [6]:
from sklearn.linear_model import LogisticRegression, SGDClassifier 

In [7]:
LRClassifier = SklearnClassifier(LogisticRegression())

In [ ]:
LR_classifier = LRClassifier.train(features_set_training)

In [ ]:
print("Logistic Regression Classifier accuracy : " , (nltk.classify.accuracy(LR_classifier,features_set_test) * 100),"%")

In [ ]:
SDGClassifier = SklearnClassifier(SGDClassifier())
SDG_classifier = SDGClassifier.train(features_set_training)
print("Logistic Regression Classifier accuracy : " , (nltk.classify.accuracy(SDG_classifier,features_set_test) * 100),"%")

# Support Vector Machine Classifier

In [ ]:
 from sklearn.svm import SVC, LinearSVC, NuSVC

In [ ]:
SVClassifier = SklearnClassifier(SVC())
SVC_classifier = SVClassifier.train(features_set_training)
print("Logistic Regression Classifier accuracy : " , (nltk.classify.accuracy(SVC_classifier,features_set_test) * 100),"%")

In [ ]:
LSVClassifier = SklearnClassifier(LinearSVC())
LSVC_classifier = LSVClassifier.train(features_set_training)
print("Logistic Regression Classifier accuracy : " , (nltk.classify.accuracy(LSVC_classifier,features_set_test) * 100),"%")

In [ ]:
NSVClassifier = SklearnClassifier(NuSVC())
NSVC_classifier = NSVClassifier.train(features_set_training)
print("Logistic Regression Classifier accuracy : " , (nltk.classify.accuracy(NSVC_classifier,features_set_test) * 100),"%")